## Simpy tutorial

### Car

#### 1 process

- function car is a generator
- It has two stages: parking and driving
- a timeout event describes the end of an event

In [3]:
import simpy

In [4]:
def car(env):
    while True:
        print('Start parking at %d' % env.now)
        parking_duration = 5
        yield env.timeout(parking_duration)

        print('Start driving at %d' % env.now)
        trip_duration = 2
        yield env.timeout(trip_duration)

In [8]:

env = simpy.Environment()
env.process(car(env))

env.run(until=15)

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14


- env: *Environment*
- car(env): *Generator function*
- env.process(car(env)): *Process object*
- env.run(until=15): *Runs process for 15s*

#### waiting for a process

In [19]:
class Car(object):
    def __init__(self, env):
        self.env = env
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())
        
    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            # We yield the process that process() returns
            # to wait for it to finish
            yield self.env.process(self.charge(charge_duration))
            
            # The charge process has finished
            print('charging finished at {:.0f}'.format(self.env.now))
            restart_duration = 1
            yield self.env.timeout(restart_duration)
            
            # The driving can start
            print('Start driving at %d' % self.env.now + '\n')
            trip_duration = 2
            yield self.env.timeout(trip_duration)
            
    def charge(self, duration):
        yield self.env.timeout(duration)

In [38]:
env = simpy.Environment()
car = Car(env)
env.run(until=15)

Start parking and charging at 0
charging finished at 5
Start driving at 6

Start parking and charging at 8
charging finished at 13
Start driving at 14



- self.charge is a process (charging)
- run has three stages: charge, charge finished, driving

#### Interrupt a process

In [60]:
class Car(object):
    def __init__(self, env):
        self.env = env
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())
        
    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            # We yield the process that process() returns
            # to wait for it to finish
            
            try:
                yield self.env.process(self.charge(charge_duration))
            except simpy.Interrupt:
                print('Was interrupted at {:.0f}. Hope, the battery is full enough ...'.format(env.now))
            
            # The charge process has finished
            print('charging finished at {:.0f}'.format(self.env.now))
            restart_duration = 1
            yield self.env.timeout(restart_duration)
            
            # The driving can start
            print('Start driving at %d' % self.env.now + '\n')
            trip_duration = 2
            yield self.env.timeout(trip_duration)
            
    def charge(self, duration):
        yield self.env.timeout(duration)

In [63]:
def driver(env, car):
    while env.now <1:
        yield env.timeout(3)
        car.action.interrupt()

In [64]:
env = simpy.Environment()
car = Car(env)
env.process(driver(env, car))
env.run(until=15)

Start parking and charging at 0
Was interrupted at 3. Hope, the battery is full enough ...
charging finished at 3
Start driving at 4

Start parking and charging at 6
charging finished at 11
Start driving at 12

Start parking and charging at 14


#### shared resource

In [69]:
def car(env, name, bcs, driving_time, charge_duration):
    # Simulate driving to Charging Station
    yield env.timeout(driving_time)
    
    # Request one of it's charging stations
    print('car {} is arriving at {:.0f}'.format(name, env.now))
    with bcs.request() as req:
        yield req
        
        # Charge the battery
        print('starting to charge car {} at {}'.format(name, env.now))
        yield env.timeout(charge_duration)
        print('car {} finished charging at {}'.format(name, env.now))
    

In [72]:
env = simpy.Environment()
bcs = simpy.Resource(env, capacity=2)
env.process(car(env, "01", bcs, 2, 5))
env.process(car(env, "02", bcs, 2, 5))
env.process(car(env, "03", bcs, 2, 5))
env.process(car(env, "04", bcs, 2, 5))
env.run(until=15)

car 01 is arriving at 2
car 02 is arriving at 2
car 03 is arriving at 2
car 04 is arriving at 2
starting to charge car 01 at 2
starting to charge car 02 at 2
car 01 finished charging at 7
car 02 finished charging at 7
starting to charge car 03 at 7
starting to charge car 04 at 7
car 03 finished charging at 12
car 04 finished charging at 12
